In [1]:
import pandas as pd
import numpy as np
GBQ_PROJECT_ID = '620265099307'

In [2]:
## SQL query to extract additonal costs of NHS price concessions  
q='''
  #this first query is for selecting data for prescribing data already available to calculate concession impact
SELECT
  ncso.date AS month,
  rx.pct,
  ccg.org_type,
  ccg.name,
  rx.bnf_name,
  rx.bnf_code,
  SUM(quantity) AS quantity,
  SUM(quantity*ncso.add_cost_per_qty*0.928) AS add_cost #This is to adjust for a national average discount of 7.2%
FROM
  `hscic.normalised_prescribing_standard` AS rx #this is our standard prescribing file with historical data
JOIN
  `dmd.ncso_calculator` AS ncso #this prepopulated data on impact of price concessions
ON
  rx.bnf_code = ncso.bnf_code #this is joining medicines info from our prescribing file to price concession data
  AND rx.month = ncso.date #this ensures it joins on the correct date
JOIN
  hscic.ccgs AS ccg #this joins to our CCG organisational data
ON
  rx.pct = ccg.code
GROUP BY
  rx.pct,
  ccg.org_type,
  ccg.name,
  rx.bnf_code,
  rx.bnf_name,
  ncso.date,
  ncso.add_cost_per_qty
UNION ALL
  #this attaches the query below which estimates impact of price concessions where prescribing data is not yet available i.e. a projection
SELECT
  ncso.date,
  rx.pct,
  ccg.org_type,
  ccg.name,
  rx.bnf_name,
  rx.bnf_code,
  SUM(quantity) AS quantity,
  SUM(quantity*ncso.add_cost_per_qty*0.928) AS add_cost
FROM
  `hscic.normalised_prescribing_standard_latest_month` AS rx #this uses a dynamic view which always shows the latest month of prescribing data available to project price concessions
JOIN
  `dmd.ncso_calculator` AS ncso
ON
  rx.bnf_code = ncso.bnf_code
JOIN
  hscic.ccgs AS ccg
ON
  rx.pct = ccg.code
WHERE
  #this identifies which months we have price concession data for but not prescribing data
  ncso.date>(
  SELECT
    MAX(rx1.month)
  FROM
    hscic.normalised_prescribing_standard AS rx1)
GROUP BY
  rx.pct,
  ccg.org_type,
  ccg.name,
  rx.bnf_code,
  rx.bnf_name,
  ncso.date,
  ncso.add_cost_per_qty'''
   

    
all_ncso = pd.read_gbq(q, GBQ_PROJECT_ID, verbose = False, dialect = 'standard')

C:\Users\bmackenna\Anaconda3\lib\site-packages\pandas_gbq\gbq.py:556: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  "verbosity", FutureWarning, stacklevel=1)


In [3]:
## Selecting nonCCG organisations to work 
nonccg_ncso  = all_ncso.loc[all_ncso.org_type !='CCG']
nonccg_ncso.head()

,month,pct,org_type,name,bnf_name,bnf_code,quantity,add_cost
3,2015-08-01,RM1,Unknown,None,Diclofenac Sod_Tab E/C 50mg,1001010C0AAAEAE,287,18.072800
17,2015-08-01,NDA,Unknown,None,Digoxin_Tab 62.5mcg,0201010F0AAADAD,28,1.531200
22,2015-08-01,RY5,Unknown,None,Diclofenac Sod_Tab E/C 50mg,1001010C0AAAEAE,15,0.944571
23,2015-08-01,NL1,Unknown,None,Diclofenac Sod_Tab E/C 50mg,1001010C0AAAEAE,21,1.322400
35,2015-08-01,AD9,Unknown,None,Chlorphenamine Mal_Oral Soln 2mg/5ml,0304010G0AAABAB,300,1.688960


# 2018 Impact - all months, all orgs, all meds

In [4]:
## ensuring the format is consistent for pounds and pence
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [5]:
## Restricting data to 2018 calendar year
nonccg_ncso_thisyear = nonccg_ncso.loc[(nonccg_ncso["month"]>="2018-01-01") & (nonccg_ncso["month"]<="2018-12-01")]
nonccg_ncso_thisyear.head()

,month,pct,org_type,name,bnf_name,bnf_code,quantity,add_cost
2503,2018-08-01,NQH,Unknown,None,Fexofenadine HCl_Tab 180mg,0304010E0AAABAB,1012,7.83
2511,2018-08-01,NI3,Unknown,None,Buprenorphine_Tab Subling 2mg S/F,0410030A0AAADAD,17593,8979.47
2534,2018-08-01,216,Unknown,None,Buprenorphine_Tab Subling 2mg S/F,0410030A0AAADAD,1431,730.38
2541,2018-08-01,NKB,Unknown,None,Sumatriptan_Tab 100mg,0407041T0AAACAC,24,2.38
2545,2018-08-01,723,Unknown,None,Buprenorphine_Tab Subling 2mg S/F,0410030A0AAADAD,2485,1268.34


In [6]:
## Total cost impact of price consessions this year to nonCCG organisations. This will return an "unknown name"
nonccg_ncso_thisyear.groupby('org_type').sum().sort_values(by = 'add_cost', ascending = False)

,quantity,add_cost
org_type,,
Unknown,9290025,5632136.09


In [7]:
## nonccg organisations grouped by indivual preparations
nonccg_ncso_thisyear.groupby('bnf_name').sum().sort_values(by = 'add_cost', ascending = False)

,quantity,add_cost
bnf_name,,
Buprenorphine_Tab Subling 8mg S/F,2662809,3646351.27
Buprenorphine_Tab Subling 2mg S/F,3450504,1823929.34
Phenoxymethylpenicillin_Soln 125mg/5ml,561600,18634.74
Phenoxymethylpenicillin_Soln 250mg/5ml,331900,9179.18
Naproxen_Tab 500mg,53030,8421.66
Diamorph HCl_Inj 30mg Amp,7637,7172.18
Metronidazole_Tab 400mg,43703,6787.20
Risperidone_Tab 500mcg,69255,6332.99
Diamorph HCl_Inj 10mg Amp,7622,6285.61


In [8]:
## monthly impact of price concessions for nonCCG organisations
nonccg_ncso_thisyear.groupby('month').sum()

,quantity,add_cost
month,,
2018-01-01,222494,12327.17
2018-02-01,526990,20018.37
2018-03-01,653831,23690.55
2018-04-01,988646,44416.92
2018-05-01,972794,870661.19
2018-06-01,795721,564720.51
2018-07-01,829977,626957.29
2018-08-01,820334,480744.40
2018-09-01,763815,738880.79


### Impact on individual organisations

In [9]:
## assess impact on each organisation "PCT"
nonccg_ncso_orgs = nonccg_ncso_thisyear.groupby('pct').sum().sort_values(by = 'add_cost', ascending = False)

In [10]:
## import councils from NHS Digital ODS codes
df_councils = pd.read_csv(r'C:\Users\bmackenna\Documents\GitHub\nonCCG Concessions\Lauth.csv')


In [11]:
## import "independent providers" from NHS Digital ODS codes
df_indprov = pd.read_csv(r'C:\Users\bmackenna\Documents\GitHub\nonCCG Concessions\ephp.csv')
df_councils['pct'] = df_councils['pct'].astype('str')


In [12]:
## import hospitals from NHS Digital ODS codes
df_hosp = pd.read_csv(r'C:\Users\bmackenna\Documents\GitHub\nonCCG Concessions\etr.csv')


In [13]:
importmerge_orgnames_part1 = nonccg_ncso_orgs.reset_index().merge(df_councils[['pct','name']],  how = "outer", on = 'pct')


In [14]:
importmerge_orgnames_part2 = importmerge_orgnames_part1.merge(df_indprov[['pct','name']],  how="outer", on='pct', suffixes=("_council","_independent_provider"))


In [15]:
fullmerge = importmerge_orgnames_part2.merge(df_hosp[['pct','name']],  how="outer", on='pct')


In [16]:
fullmerge['purchaser_name'] = fullmerge['name_council'].fillna('') + fullmerge['name_independent_provider'].fillna('') + fullmerge['name'].fillna('')


In [17]:
purchaser_list = fullmerge.drop('name_council', axis=1) ####getting multiple errors when trying to drop all at once


In [18]:
purchaser_list1 = purchaser_list.drop('name', axis=1)

In [19]:
purchaser_list2 = purchaser_list1.drop('name_independent_provider', axis=1)
purchaser_list2[['purchaser_name', 'pct', 'add_cost']]

,purchaser_name,pct,add_cost
0,"CHANGE, GROW, LIVE",NMS,2091871.32
1,TURNING POINT,NKI,582024.50
2,ADDACTION,NI3,226144.28
3,LEEDS CITY COUNCIL,212,181742.50
4,"NORTHUMBERLAND, TYNE AND WEAR NHS FOUNDATION T...",RX4,174186.77
5,MIDDLESBROUGH COUNCIL,112,164438.04
6,NEWCASTLE-UPON-TYNE CITY COUNCIL,107,124219.57
7,SPECTRUM COMMUNITY HEALTH - CIC,NL1,122254.48
8,THE FORWARD TRUST,DG3,116849.06
9,STOCKTON-ON-TEES BOROUGH COUNCIL,114,113859.05
